# Урок 7. Сверточные нейронные сети для анализа текста

## Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Обучить conv сеть для классификации. Рассмотреть 2-а варианта сеточек<br>
    1.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/<br>
    1.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)<br>

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше.

## Полезные ссылки
 - [Использование предобученных эмбедингов1](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)
 - [Использование предобученных эмбедингов2](https://keras.io/examples/nlp/pretrained_word_embeddings/)
 - [предобученные модели word2vec fasttext (смотреть Статические модели)](https://rusvectores.org/ru/models/)

## Решение.

### Подготовка данных.

In [4]:
# !pip install xlrd
# !pip install stop_words

In [1]:
import os
import pandas as pd

In [5]:
df = pd.read_excel('отзывы за лето.xls')

In [7]:
df = df[['Rating','Content']]
df.head()

,Rating,Content
0,5,It just works!
1,4,В целом удобноное приложение...из минусов хотя...
2,5,Отлично все
3,5,Стал зависать на 1% работы антивируса. Дальше ...
4,5,"Очень удобно, работает быстро."
...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра..."
20655,5,Ок
20656,4,Доволен
20657,1,"Песопаснасть, рут ни нужын"


In [8]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Установи порог отзывов 3 бала. если значение выше порога отзыв положительный. Если ниже- отрицательный.

In [9]:
df['Rating'] = df['Rating'].apply(lambda x: 1 if x > 3 else 0)

In [10]:
df['Rating'].value_counts()

1    16724
0     3935
Name: Rating, dtype: int64

### Предобработка текста.

Т.к. в работе мы будем использовать предобученную модель **rusvectores**, то предобработку текста выполним в используя пример с этого сайта.

In [11]:
max_words = 1000
max_sentence_len = 100
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [13]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from utils import apostrophe_dict, emoticon_dict, short_word_dict  # см. файл utils.py
from  rus_preprocessing_udpipe import process

import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline
from tqdm import tqdm

tqdm.pandas()

/home/sergey/anaconda3/envs/deeppavlov_env/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [15]:
df.head(5)

,Rating,Content
0,1,It just works!
1,1,В целом удобноное приложение...из минусов хотя...
2,1,Отлично все
3,1,Стал зависать на 1% работы антивируса. Дальше ...
4,1,"Очень удобно, работает быстро."
...,...,...
20654,0,"Ну и шляпа,с роот правами бесполезная прога,ра..."
20655,1,Ок
20656,1,Доволен
20657,0,"Песопаснасть, рут ни нужын"


In [16]:
# Теперь повторим это для всех записей.
def replase_words(text,dict_=emoticon_dict): 
    text = str(text)
    output = ''
    for word in text.split(' '): # не будем делить текст на части будем искать подстроку в строке. это касается только смайликов.
        word = word.strip()
        if word in dict_.keys(): 
            output += ' ' + dict_[word]
        else:
            output += ' ' + wordmodel
    return output

In [17]:
# txt = replase_words(txt, emoticon_dict)
df['Content'] = df['Content'].apply(replase_words)

Загрузка предобученной модели для NER.

In [18]:
udpipe_model_url = "https://rusvectores.org/static/models/udpipe_syntagrus.model"
udpipe_filename = udpipe_model_url.split("/")[-1]

if not os.path.isfile(udpipe_filename):
    print("UDPipe model not found. Downloading...", file=sys.stderr)
    wget.download(udpipe_model_url)

# print("\nLoading the model...", file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(
    model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu"
)


Loading the model...


Проверка

In [19]:
# print("Processing input...", file=sys.stderr)
# for input_line in sys.stdin:
#     break
res = 'очень странный текст о котором мы хотим поговорить'#unify_sym(input_line.strip())
print(res)
output = process(process_pipeline, text=res)
print(" ".join(output))

очень странный текст о котором мы хотим поговорить
очень_ADV странный_ADJ текст_NOUN о_ADP который_PRON мы_PRON хотеть_VERB говорить_VERB


Processing input...


In [20]:
# Попробуем тоже самое сделать и для нашего датасета.
df['Content'] = df['Content'].progress_apply(lambda x:" ".join(process(process_pipeline, text=x)))

100%|██████████| 20659/20659 [01:53<00:00, 182.80it/s]


Раделим DataFrame на ренировочную и тестовую выборки !!! 

In [26]:
df_train, df_test = train_test_split(df,test_size=0.3)

In [27]:
train_corpus = " ".join(df_train["Content"])

In [29]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

/home/sergey/anaconda3/envs/deeppavlov_env/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/sergey/anaconda3/envs/deeppavlov_env/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in Nu

In [30]:
tokens_filtered = [word for word in tokens]# if word.isalnum()]
tokens_filtered

['ужас.ва_PROPN',
 'что_SCONJ',
 'творите_SYM',
 'если_SCONJ',
 'рано_ADV',
 'хоть_ADV',
 'как_SCONJ',
 'то_PRON',
 'определять_VERB',
 'сканировать_VERB',
 'штрих_ADJ',
 'код_NOUN',
 'то_SCONJ',
 'сейчас_ADV',
 'вообще_ADV',
 'ни_PART',
 'находить_VERB',
 'свой_DET',
 'получатель_NOUN',
 'невозможно.вводить_VERB',
 'инн_NOUN',
 'бик_NOUN',
 'и_CCONJ',
 'все_PRON',
 'что_PRON',
 'необходимый_ADJ',
 'этот_PRON',
 'же_PART',
 'сколько_ADV',
 'время_NOUN',
 'оплата_NOUN',
 'жкй_ADJ',
 'становиться_VERB',
 'просто_PART',
 'невыносимый_ADJ',
 'вы_PRON',
 'предлогать_VERB',
 'вернуться_VERB',
 'снова_ADV',
 'в_ADP',
 'филиал_NOUN',
 'для_ADP',
 'оплата_NOUN',
 'жкх_PROPN',
 'и_CCONJ',
 'прочий_ADJ',
 'услуга_NOUN',
 'ваш_DET',
 'антивирус_NOUN',
 'находить_VERB',
 'вредоносный_ADJ',
 'файл_NOUN',
 'и_CCONJ',
 'ограничивать_VERB',
 'право_NOUN',
 'предлагать_VERB',
 'удалять_VERB',
 'но_CCONJ',
 'не_PART',
 'удалять_VERB',
 'другой_ADJ',
 'антивирусый_NOUN',
 'ничто_PRON',
 'не_PART',
 'наход

Отфильтруем данные и соберём в корпус N наиболее частых токенов

In [31]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [32]:
tokens_filtered_top[:10]

['приложение_NOUN',
 'не_PART',
 'удобный_ADJ',
 'и_CCONJ',
 'очень_ADV',
 'все_PRON',
 'в_ADP',
 'я_PRON',
 'на_ADP',
 'работать_VERB']

In [33]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [34]:
vocabulary

{'приложение_NOUN': 1,
 'не_PART': 2,
 'удобный_ADJ': 3,
 'и_CCONJ': 4,
 'очень_ADV': 5,
 'все_PRON': 6,
 'в_ADP': 7,
 'я_PRON': 8,
 'на_ADP': 9,
 'работать_VERB': 10,
 'с_ADP': 11,
 'отличный_ADJ': 12,
 'хороший_ADJ': 13,
 ',': 14,
 'нравиться_VERB': 15,
 'раз_NOUN': 16,
 'по_ADP': 17,
 'телефон_NOUN': 18,
 'что_SCONJ': 19,
 'но_CCONJ': 20,
 'а_CCONJ': 21,
 'после_ADP': 22,
 'мочь_VERB': 23,
 'спасибо_PROPN': 24,
 'это_PRON': 25,
 'что_PRON': 26,
 'x_NUM': 27,
 'обновление_NOUN': 28,
 'весь_DET': 29,
 'при_ADP': 30,
 'супер_NOUN': 31,
 'за_ADP': 32,
 'пользоваться_VERB': 33,
 'так_ADV': 34,
 'быть_AUX': 35,
 'и_PART': 36,
 'только_PART': 37,
 'банк_NOUN': 38,
 'антивирус_NOUN': 39,
 'быть_VERB': 40,
 'быстро_ADV': 41,
 'весь_PRON': 42,
 'делать_VERB': 43,
 'устраивать_VERB': 44,
 'нет_VERB': 45,
 'пароль_NOUN': 46,
 'у_ADP': 47,
 'вы_PRON': 48,
 'вход_NOUN': 49,
 'как_SCONJ': 50,
 'он_PRON': 51,
 'карта_NOUN': 52,
 'прошивка_NOUN': 53,
 'заходить_VERB': 54,
 'этот_DET': 55,
 'для_ADP'

In [35]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens ] #if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [36]:
x_train = np.asarray([text_to_sequence(text, max_sentence_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_sentence_len) for text in df_test["Content"]], dtype=np.int32)
# x_val = np.asarray([text_to_sequence(text, max_sentence_len) for text in df_val["Content"]], dtype=np.int32)

# Обучим нейронную сеть.

In [38]:
import numpy as np
import tensorflow.keras
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D,GlobalMaxPooling1D

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.callbacks import EarlyStopping  
from sklearn.metrics import accuracy_score 

 __1.1__ Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/<br>

Загрузим архим с предобученным Embedding-ом.

In [55]:
# !wget "http://vectors.nlpl.eu/repository/20/180.zip"
# !unzip "180.zip"

In [56]:
# import numpy as np
# !pip install gensim
# !pip install numpy==1.20.0
# !pip install pycocotools==2.0.0

Поскольку обучение и загрузка моделей могут занимать продолжительное время, иногда бывает полезно вести лог событий. Для этого используется стандартная питоновская библиотека logging.

In [64]:
import sys
import gensim
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [65]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('180/model.bin', binary=True)

2021-09-03 13:22:57,842 : INFO : loading projection weights from 180/model.bin
2021-09-03 13:23:00,461 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (189193, 300) matrix of type float32 from 180/model.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2021-09-03T13:23:00.461599', 'gensim': '4.1.0', 'python': '3.7.11 (default, Jul 27 2021, 14:32:16) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.0-81-generic-x86_64-with-debian-bullseye-sid', 'event': 'load_word2vec_format'}


In [120]:
pretrained_weights = word_model.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print(f'vocab_size: {vocab_size}, emdedding_size: {emdedding_size}')

vocab_size: 189193, emdedding_size: 300


In [121]:
train_count = len(df_train)
test_count = len(df_test)

In [123]:
df_train.head()

,Rating,Content,Text_len
16891,0,ужас.ва_PROPN что_SCONJ творите_SYM если_SCONJ...,530
16791,1,ваш_DET антивирус_NOUN находить_VERB вредоносн...,389
15102,0,весь_PRON классный_ADJ спасибо_NOUN,35
4857,1,суппер_NOUN удобный_ADJ,23
7806,1,весь_DET супер_NOUN,19
...,...,...,...
16296,1,использовать_ADJ почти_ADV год_NOUN смущать_VE...,124
11358,1,вполне_ADV удобный_ADJ весь_DET необходимый_AD...,73
5376,1,после_ADP последний_ADJ обновление_NOUN приход...,434
4589,0,хорошо_ADV дорабатывать_VERB программа_NOUN мо...,70


In [124]:
def word2idx(word):
  return word_model.key_to_index[word] # если нет такого токена. то возвращаем пустую строку. или вообще не добавляем элемент.
def idx2word(idx):
  return word_model.index_to_key[idx]


train_y = tensorflow.keras.utils.to_categorical(df_train["Rating"], num_classes)
train_x = np.zeros([train_count, max_sentence_len], dtype=np.int32)

for i, sentence in enumerate(df_train['Content']):
  tokens = word_tokenize(sentence)
  for t, word in enumerate(tokens):
    try:
      train_x[i, t] = word2idx(word) # отбрасывам все токены, которые отсутствуют в предобученной модели.
    except:
      continue

print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

train_x shape: (14461, 100)
train_y shape: (14461, 2)


In [125]:
x = df_test['Content']
y = df_test['Rating']

test_y = tensorflow.keras.utils.to_categorical(df_test["Rating"], num_classes)
test_x = np.zeros([test_count, max_sentence_len], dtype=np.int32)

for i, sentence in enumerate(df_test['Content']):
  tokens = word_tokenize(sentence)
  for t, word in enumerate(tokens):
    try:
      test_x[i, t] = word2idx(word)
    except:
      continue

print('test_x shape:', test_x.shape)
print('test_y shape:', test_y.shape)

test_x shape: (6198, 100)
test_y shape: (6198, 2)


In [126]:
train_y

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [127]:
train_x[0]

array([     0,      0,      0,      0,    323,    280,      0,      0,
          434,  55917,  32506,   7802,      0,     83,    191,      0,
           98,      0,  20036,      0,  90895, 136779,      0,      0,
            0,    234,      0,      0,    220,     12,   3501,      0,
           31,      0,   5231,      0,      0,    278,    202,      0,
         7222,      0,   3501,   9751,      0,    682,   1344,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0], dtype=int32)

In [128]:
word_model.vectors[0]

array([[ 1.0967804 , -2.2944486 ,  1.9791678 , ...,  0.90631187,
         3.2207503 , -0.0495954 ],
       [ 0.56827927, -0.3209115 , -0.01912439, ...,  0.20497806,
         0.17032085,  0.20717178],
       [ 2.639508  , -3.596937  , -0.13203041, ...,  1.6058776 ,
         0.8480653 , -0.4020421 ],
       ...,
       [ 0.14449295, -0.11888789, -0.1052237 , ..., -0.19022939,
        -0.19688326,  0.22379388],
       [ 0.07240537,  0.18123615, -0.1760937 , ..., -0.31739685,
         0.12159412,  0.0491648 ],
       [-0.0882876 , -0.08472444, -0.36188012, ...,  0.0580937 ,
        -0.1582485 , -0.18358405]], dtype=float32)

In [129]:
# Создадим слой Embedding.
embedidding = Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights], trainable=False)

In [131]:
model = Sequential()
model.add(embedidding)
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [134]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [135]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(train_x, train_y,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Train on 13014 samples, validate on 1447 samples
Epoch 1/20
13014/13014 [==============================] - 14s 1ms/sample - loss: 0.3625 - acc: 0.8259 - val_loss: 0.2854 - val_acc: 0.8687
Epoch 2/20
13014/13014 [==============================] - 10s 794us/sample - loss: 0.2495 - acc: 0.8910 - val_loss: 0.2635 - val_acc: 0.8673
Epoch 3/20
13014/13014 [==============================] - 11s 868us/sample - loss: 0.2197 - acc: 0.9045 - val_loss: 0.2583 - val_acc: 0.8749
Epoch 4/20
13014/13014 [==============================] - 10s 762us/sample - loss: 0.1993 - acc: 0.9179 - val_loss: 0.2427 - val_acc: 0.8804
Epoch 5/20
13014/13014 [==============================] - 9s 720us/sample - loss: 0.1807 - acc: 0.9292 - val_loss: 0.2447 - val_acc: 0.8798


In [136]:
results = model.predict(test_x, batch_size=batch_size, verbose=1)

6198/6198 [==============================] - 1s 217us/sample


In [137]:
pred_y = np.argmax(results,axis = 1)

In [139]:
rusvectores_accuracy = accuracy_score(np.argmax(test_y,axis = 1), pred_y)
rusvectores_accuracy

0.8994837044207808

__1.2__ Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)<br>

In [ ]:
num_classes = 2
y_train = tensorflow.keras.utils.to_categorical(df_train["Rating"], num_classes)
y_test = tensorflow.keras.utils.to_categorical(df_test["Rating"], num_classes)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sentence_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

In [ ]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

In [ ]:
y_pred = np.argmax(results,axis = 1)

In [ ]:
simple_accuracy = accuracy_score(df_test['Rating'].to_numpy(), y_pred)
simple_accuracy

In [ ]:
print('Результат:')
print(f'1. rusvectores: {rusvectores_accuracy}, 2. simple: {simple_accuracy}  ')

## Вывод -- 
    Как видно, Модель с предобученным эмбедингом показала лучший результат.


TODO: Предобработка текста в соответствии с рекомендациями rusvectores показала результат хуже чем ту что мы делали на уроках. см. файл HW_NLP_Lesson_7_1. Когда будет время. необходимо будет переделать предобработку.